# 3_create_ML_dataset_using_Dataflow

In this notebook, we'll use Cloud Dataflow to preprocess the entire dataset from BigQuery and create CSV files for the training/evaluation datasets for operationalizing the ML model later.

Some benefits of using Dataflow:
- Dataflow sets itself apart as a platform for data transformations because it is a serverless, fully managed offering from Google that allows you to execute Data Processing Pipelines at scale.
- Dataflow executes our code using the Apache Beam API. Apache Beam supports both batch and streaming processing using the same pipeline code.
- Dataflow changes the amount of compute resources, the number of servers that will run your pipeline elastically, all depending on the amount of data that your pipeline needs to process.


## Import necessary libraries & Set up environment variables 

In [ ]:
!pip install --user google-cloud-bigquery==1.25.0

In [ ]:
!pip install --user apache-beam[interactive]==2.24.0

In [1]:
import os
import pandas
import datetime
from google.cloud import bigquery
import apache_beam as beam
print(beam.__version__)
import tensorflow as tf
print(tf.__version__)

2.24.0
2.3.4


In [2]:
import hashlib
import copy
import shutil
import subprocess

In [36]:
PROJECT = "predict-babyweight-10142021"
BUCKET = PROJECT
REGION = "us-central1"

os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET 
os.environ["REGION"] = REGION

In [5]:
%%bash
if ! gsutil ls|grep -q gs://${BUCKET}/; then
    gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Call BigQuery and examine in the dataframe

In [4]:
bq = bigquery.Client()

In [5]:
query_2003 = """
SELECT
    weight_pounds,
    CAST(is_male AS STRING) AS is_male,
    mother_age,
    plurality,
    gestation_weeks,
    IFNULL(CAST(cigarette_use AS STRING), "Unknown") AS cigarette_use,
    IFNULL(CAST(alcohol_use AS STRING), "Unknown") AS alcohol_use,
    year,
    month,
    COALESCE(wday, day, 0) AS date,
    IFNULL(state, "Unknown") AS state,
    IFNULL(mother_birth_state, "Unknown") AS mother_birth_state
FROM
    publicdata.samples.natality
WHERE
    year > 2002
    AND weight_pounds > 0
    AND mother_age > 0
    AND plurality > 0
    AND gestation_weeks > 0
"""

query_2003_with_hash_vals=f"""
SELECT
    weight_pounds,
    is_male,
    mother_age,
    plurality,
    gestation_weeks,
    cigarette_use,
    alcohol_use,
    ABS(FARM_FINGERPRINT(
        CONCAT(
            CAST(year AS STRING),
            CAST(month AS STRING),
            CAST(date AS STRING),
            CAST(state AS STRING),
            CAST(mother_birth_state AS STRING)
        )
    )) AS hash_values
FROM
    ({query_2003})
"""


Let's find how many records from the `query_2003` call

In [6]:
query_2003_count = f"SELECT COUNT(*) FROM ({query_2003})"
df_count = bq.query(query_2003_count).to_dataframe()
num_records = df_count['f0_'][0]
print('*** Number of all records found:',num_records)

*** Number of all records found: 25037335


View the `query_2003_with_hash_vals` result

In [7]:
df_limit_100 = bq.query(query_2003_with_hash_vals + "LIMIT 100").to_dataframe()
df_limit_100.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,cigarette_use,alcohol_use,hash_values
0,7.264232,True,22,1,39,Unknown,Unknown,8045173873969881371
1,6.563162,True,38,2,39,Unknown,Unknown,16293285635216904
2,7.209116,False,18,1,41,false,false,4931362078050829102
3,6.316244,True,25,1,38,Unknown,Unknown,7429616553140235181
4,7.603743,False,23,1,39,Unknown,Unknown,1781791737502110095


## Create ML dataset using Dataflow
Let's use Cloud Dataflow to read in the BigQuery data, do some preprocessing, and write it out as CSV files.

For the preprocessing, we'll do:

- Modify plurality field to be a string where [1, 2, 3, 4, 5] becomes ["Single(1)", "Twins(2)", "Triplets(3)", "Quadruplets(4)", "Quintuplets(5)"]
- Augment our dataset with our three simulated babyweight data by:
    - setting all gender information to Unknown and setting plurality of all non-single births to Multiple(2+),
    - setting cigarette_use information to Unknown,
    - setting alcohol_use information to Unknown.
    
Fitst, let's define a function to create line(s) of CSV input from columns called by BigQuery

In [24]:
def to_csv(rowdict):
    import hashlib
    import copy
    
    CSV_COLUMNS = ["weight_pounds",
                   "is_male",
                   "mother_age",
                   "plurality",
                   "gestation_weeks",
                   "cigarette_use",
                   "alcohol_use"]

    # Modify plurality field
    rowdict_edited = copy.deepcopy(rowdict)
    rowdict_edited['plurality'] = ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)'][rowdict['plurality'] - 1]
    
    # Clone data and mask certain columns to simulate lack of ultrasound
    no_ultrasound = copy.deepcopy(rowdict_edited)
    no_ultrasound['is_male'] = 'Unknown'
    no_ultrasound['plurality'] = 'Multiple(2+)' if rowdict['plurality'] > 1 else 'Single(1)'
    
    # Write out rows for each input row
    for result in [rowdict_edited, no_ultrasound]:
        data = ','.join([str(result[k]) if k in result else 'None' for k in CSV_COLUMNS])
        #key = hashlib.sha224(data.encode('utf-8')).hexdigest()  # hash the columns to form a key
        #yield str(f'{data},{key}')
        yield str(f'{data}')

Dataflow job will start with a selection BigQuery, converting it to CSV, and writing the output as CSV files.

In [28]:
def preprocess(in_test_mode=True):
    import shutil, os, subprocess
    
    HEADER = 'weight_pounds,is_male,mother_age,plurality,gestation_weeks,cigarette_use,alcohol_use'
    job_name = 'preprocess-babyweight-features' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')

    if in_test_mode:
        print('Launching local job ... hang on')
        OUTPUT_DIR = './datasets_preprocessed_Dataflow_test_mode'
        shutil.rmtree(OUTPUT_DIR, ignore_errors=True)
        os.makedirs(OUTPUT_DIR)
    else:
        print(f'Launching Dataflow job {job_name} ... hang on')
        OUTPUT_DIR = f'gs://{BUCKET}/datasets_preprocessed_Dataflow/'
        try:
            subprocess.check_call(f'gsutil -m rm -r {OUTPUT_DIR}'.split())
        except:
            pass

    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': job_name,
        'project': PROJECT,
        'region': REGION,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True,
    }
    
    opts = beam.pipeline.PipelineOptions(flags = [], **options)
    if in_test_mode:
        RUNNER = 'DirectRunner'
    else:
        RUNNER = 'DataflowRunner'
        
    p = beam.Pipeline(RUNNER, options = opts)
    
    query = query_2003_with_hash_vals
    if in_test_mode:
        query = query + ' LIMIT 100' 
    
    for step in ['train', 'eval', 'test']:
        if step == 'train':
            selquery = f'SELECT * FROM ({query}) WHERE MOD(hash_values, 100) < 80'
        elif step == 'eval':
            selquery = f'SELECT * FROM ({query}) WHERE MOD(hash_values, 100) >= 80 AND MOD(hash_values, 100) < 90'
        else:
            selquery = f'SELECT * FROM ({query}) WHERE MOD(hash_values, 100) > 90'
        (p 
         | f'{step}_read' >> beam.io.Read(beam.io.BigQuerySource(query = selquery, use_standard_sql = True))
         | f'{step}_csv' >> beam.FlatMap(to_csv)
         | f'{step}_out' >> beam.io.Write(beam.io.WriteToText(os.path.join(OUTPUT_DIR, f'{step}.csv'),header=HEADER))
        )
    job = p.run()
    
    if in_test_mode:
        job.wait_until_finish()
        print("Done!")

First, let's test the `preprocess` function locally to see if it works correctly

In [29]:
preprocess(in_test_mode = True)

Launching local job ... hang on


Done!


In [30]:
!head -5 datasets_preprocessed_Dataflow_test_mode/train*

weight_pounds,is_male,mother_age,plurality,gestation_weeks,cigarette_use,alcohol_use
6.8673994613,False,27,Single(1),35,Unknown,Unknown
6.8673994613,Unknown,27,Single(1),35,Unknown,Unknown
5.68572173698,False,27,Single(1),36,false,false
5.68572173698,Unknown,27,Single(1),36,false,false


Once everything has run correctly, let's execute the job in Cloud Dataflow. We can monitor the running job at the Dataflow section in the GCP web console.

In [31]:
preprocess(in_test_mode = False)

Launching Dataflow job preprocess-babyweight-features-211023-045809 ... hang on


After ~13 minutes, the job has been done. At this point, we now have the training and evaluation datasets created at scale. The process is also fully automated. We can simply re-run the pipeline periodically to create a new training dataset on fresher data.

In [39]:
%%bash
gsutil ls gs://${BUCKET}/datasets_preprocessed_Dataflow/

gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00000-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00001-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00002-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00003-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00004-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/eval.csv-00005-of-00006
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/test.csv-00000-of-00003
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/test.csv-00001-of-00003
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/test.csv-00002-of-00003
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/train.csv-00000-of-00020
gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/train.csv-00001-of-00020
gs://predict-babyweight-101420

In [40]:
%%bash
gsutil cat gs://predict-babyweight-10142021/datasets_preprocessed_Dataflow/train.csv-00000-of-00020 |head -5

weight_pounds,is_male,mother_age,plurality,gestation_weeks,cigarette_use,alcohol_use
6.87621795178,False,18,Single(1),42,false,false
6.87621795178,Unknown,18,Single(1),42,false,false
6.81448851842,False,18,Single(1),42,Unknown,Unknown
6.81448851842,Unknown,18,Single(1),42,Unknown,Unknown


## Summary:

- Using Dataflow, we performed multiple preproseeing, modifying, and simulating data for the entire dataset and then produced CSV files for the training/evaluation datasets. 
- These files are storaged in the Cloud bucket and ready for the training a ML model at scale.

